In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch.nn.utils.prune as prune

# from data import dataloaders
# from net import resnet, resnet_for_cifar, params_to_prune
# from main import Trainer, Tester

# import os
# from tqdm import tqdm
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

cifar_pth = '/.cifar'
if os.path.exists(cifar_pth):
    os.mkdir(cifar_pth)

resnet = resnet_for_cifar()
resnet = resnet.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet.parameters(), lr=1e-3)

trainer = Trainer(optimizer=optimizer, model=resnet, loss_func=loss_func)
trainer.pruning(amnt=0.2)
# trainer.freeze_10()
# trainer.freeze_20()
# trainer.freeze_30()
tester = Tester(model=resnet, loss_func=loss_func)

train_loss_per_epoch = []
val_loss_per_epoch = []

train_acc_per_epoch = []
val_acc_per_epoch = []

loader = dataloaders()
train_dl = loader.return_traindl()
val_dl = loader.return_testdl()

best_training_acc = 0 # for a batch
best_validation_acc = 0 # for a batch

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar/cifar-10-python.tar.gz to ./cifar
Files already downloaded and verified


In [ ]:
epochs = 600
for epoch in tqdm(range(epochs)):

    train_loss_per_loader = []
    val_loss_per_loader = []

    train_acc_per_loader = []
    val_acc_per_loader = []

    for train_data, val_data in zip(train_dl, val_dl):
        train_img, train_label = train_data
        train_img, train_label = train_img.to(device), train_label.to(device)
        train_loss = trainer.train(train_img, train_label)
        train_acc = trainer.check_accuracy()

        val_img, val_label = val_data
        val_img, val_label = val_img.to(device), val_label.to(device)
        val_loss = tester.validate(val_img, val_label)
        val_acc = tester.check_accuracy()

        train_loss_per_loader.append(train_loss)
        val_loss_per_loader.append(val_loss)

        train_acc_per_loader.append(train_acc)
        val_acc_per_loader.append(val_acc)

        tester.save_model(filename='pruned_20.pth')

    train_loss_per_epoch.append(np.mean(train_loss_per_loader))
    val_loss_per_epoch.append(np.mean(val_loss_per_loader))

    avg_train_acc = np.mean(train_acc_per_loader)
    avg_val_acc = np.mean(val_acc_per_loader)

    if avg_train_acc > best_training_acc:
        best_training_acc = avg_train_acc

    if avg_val_acc > best_validation_acc:
        best_validation_acc = avg_val_acc

    train_acc_per_epoch.append(avg_train_acc)
    val_acc_per_epoch.append(avg_val_acc)

    with open('train_loss.npy', 'wb') as f:
        np.save(f, np.array(train_loss_per_epoch))
    with open('val_loss.npy', 'wb') as f:
        np.save(f, np.array(val_loss_per_epoch))

    with open('train_acc.npy', 'wb') as f:
        np.save(f, np.array(train_acc_per_epoch))
    with open('val_acc.npy', 'wb') as f:
        np.save(f, np.array(val_acc_per_epoch))

    if (epoch + 1) % 10 == 0:
        print(
            f'Training Loss: {np.mean(train_loss_per_loader):.4f} Validation Loss: {np.mean(val_loss_per_loader):.4f}')
        print(
            f'Training Acc: {best_training_acc:.2f} Validation Acc: {best_validation_acc:.2f}')
        trainer.check_sparsity()

    plt.figure(dpi=300)
    plt.tight_layout()
    plt.plot(list(range(len(train_loss_per_epoch))), train_loss_per_epoch, label='train')
    plt.plot(list(range(len(val_loss_per_epoch))), val_loss_per_epoch, label='val')
    plt.legend(loc="upper left")
    plt.savefig('loss.png')

    plt.figure(dpi=300)
    plt.tight_layout()
    plt.plot(list(range(len(train_acc_per_epoch))), train_acc_per_epoch, label='train')
    plt.plot(list(range(len(val_acc_per_epoch))), val_acc_per_epoch, label='val')
    plt.legend(loc="upper left")
    plt.savefig('acc.png')


  2%|▏         | 9/600 [01:29<1:31:34,  9.30s/it]

Training Loss: 0.6173 Validation Loss: 0.6902
Training Acc: 0.79 Validation Acc: 0.77
Sparsity of the network is :19.999998092651367 %


  2%|▏         | 10/600 [01:38<1:29:19,  9.08s/it]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:68: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  3%|▎         | 19/600 [02:56<1:25:16,  8.81s/it]

Training Loss: 0.4350 Validation Loss: 0.6464
Training Acc: 0.86 Validation Acc: 0.80
Sparsity of the network is :19.999998092651367 %


  5%|▍         | 29/600 [04:21<1:21:00,  8.51s/it]

...

 15%|█▍        | 89/600 [12:58<1:13:16,  8.60s/it]

Training Loss: 0.1096 Validation Loss: 0.7765
Training Acc: 0.97 Validation Acc: 0.82
Sparsity of the network is :19.999998092651367 %


 16%|█▋        | 99/600 [14:24<1:11:41,  8.58s/it]

Training Loss: 0.1009 Validation Loss: 0.8776
Training Acc: 0.97 Validation Acc: 0.82
Sparsity of the network is :19.999998092651367 %


 17%|█▋        | 104/600 [15:07<1:11:13,  8.62s/it]

In [3]:
t = torch.tensor([[1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5]])
t

tensor([[1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5],
        [1, 2, 3, 4, 5]])

In [4]:
t.view(-1)[::2] = 0

tensor([[0, 2, 0, 4, 0],
        [1, 0, 3, 0, 5],
        [0, 2, 0, 4, 0]])